##### Convert from the LAEA projected coordinate system (EPSG 3857) to a geographic coordinate system (EPSG 4326)

In [1]:
import os
import regex as re
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rio

In [2]:
# get all certain-type files inside a specific folder

def show_all_files(input_dir, typ='csv'):

    regexp = re.compile(fr".{typ}")
    files  = [path.name for path in os.scandir(input_dir) if path.is_file() if regexp.search(path.name)]
    
    return files


In [3]:
# Directories

root = 'C:/Users/freiste/OneDrive - Ilmatieteen laitos/Documents/IIASA YSSP 2023/02 - Data/EU/BOKU_ImprovedForestCharacteristics'

subfolders = [
    "Age",
    "Alive_Tree_Carbon",
    "Basal_Area",
    "Biomass",
    "Diameter",
    "Height",
    "Stand_Density_Index",
    "Stem_Number",
    "Tree_Species_Group",
    "Volume"
    ]




In [4]:
for sub in subfolders:
    
    folder = f'{root}/{sub}'
    
    files = show_all_files(folder, typ='nc')
    
    for file in files:
        
        FSDS = xr.open_dataset(f"{folder}/{file}")
        FSDS.close()
        
        FSDS.rio.write_crs('epsg:3857', inplace=True)
        FSDS_proj = FSDS.rio.reproject('epsg:4326')
        
        
        vars = [v for v in FSDS_proj.data_vars]
        for var in vars:
            try:
                del FSDS_proj[var].attrs['grid_mapping']
            except:pass
        
        FSDS = FSDS.rename({'x' : 'latitude', 'y' : 'longitude'})
        
        try:
            FSDS_proj.to_netcdf(f"{folder}/{file[:-3]}_EPSG4326.nc")
        except:
            print(f'ERROR with exporting file {file}')
        

In [ ]:
folder = 'C:/Users/freiste/OneDrive - Ilmatieteen laitos/Documents/IIASA YSSP 2023/02 - Data/EU/BOKU_ImprovedForestCharacteristics/Diameter'
file   = 'dbh_EPSG4326'
test = xr.open_dataset(f"{folder}/{file}.nc")
test.close()

In [ ]:
test.Band1.plot()